# Install dependencies

In [1]:
%pip install -r requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


# imports

In [2]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import os
from datetime import datetime, timedelta
import tradermade
import yfinance as yf

# Constants

In [3]:
TRADERMADE_API_KEY = os.getenv("TRADERMADE_API_KEY")
CURRENCY = "XAUUSD"
REQUIRED_COLUMNS = ['Open', 'High', 'Low', 'Close']
TICKER = "SPY"

END_DATE = datetime.now()
START_DATE = END_DATE - timedelta(days=365)
INTERVAL = "1d"

# Initialization

In [4]:
tradermade.set_rest_api_key(TRADERMADE_API_KEY)

# Get raw data

In [5]:
spy_data = yf.download(TICKER, start=START_DATE, end=END_DATE, interval=INTERVAL, auto_adjust=False, ignore_tz=True)

[*********************100%***********************]  1 of 1 completed


# Preprocessing data

## Handle MultiIndex columns

In [6]:
if isinstance(spy_data.columns, pd.MultiIndex):
    spy_data = spy_data.xs(TICKER, axis=1, level=1)
    spy_data.columns = [col.title() for col in spy_data.columns]
else:
    spy_data.columns = [col.title() for col in spy_data.columns]

## Convert required columns to numeric and drop rows with NaN

In [7]:
spy_data[REQUIRED_COLUMNS] = spy_data[REQUIRED_COLUMNS].apply(pd.to_numeric, errors='coerce')
spy_data = spy_data.dropna(subset=REQUIRED_COLUMNS)
spy_data.index = spy_data.index.tz_localize(None)

# Plot raw data

In [ ]:
fig = go.Figure(data=[go.Candlestick(
    x=spy_data.index,
    open=spy_data['Open'],
    high=spy_data['High'],
    low=spy_data['Low'],
    close=spy_data['Close'],
    name='SPY'
)])

fig.update_layout(
    title=f'SPY (S&P 500 ETF) Candlestick Chart - Last 60 Days',
    yaxis_title='Price (USD)',
    xaxis_title='Date',
    xaxis_rangeslider_visible=True, 
    template='seaborn', # other options: plotly, plotly_white, plotly_dark, simple_white
    showlegend=True,
)